<a href="https://colab.research.google.com/gist/toco-t/e87ee2ee1d9bde3565f420c55e23d487/llm_swarm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AI Agent Swarm for Code Improvement
---
**Author:** Toco Tachibana<br>
**GitHub:** https://github.com/Vero-Ventures/llm-swarm<br>
**Date created:** 2024/05/20<br>
**Last modified:** 2024/05/27<br>

## Overview

This guide provides a walkthrough for using [CrewAI](https://www.crewai.com/)'s Agents and [Ollama](ollama.coms)'s Large Language Models (LLMs) to improve Python code. LLMs are advanced artificial intelligence models that can process human language, enabling applications like code analysis and generation.

**CrewAI** is a framework designed to automate various aspects of software development and maintenance. By using intelligent agents, CrewAI can handle tasks ranging from code review to optimization, significantly reducing the time and effort required from human developers.

**Ollama** offers a suite of local Large Language Models (LLMs) that can be integrated into various applications to provide advanced natural language processing capabilities. These models can be run on local hardware, offering a high degree of customization and privacy compared to cloud-based solutions.

By the end of this guide, you will have a clear understanding of how to set up and utilize these tools to enhance your codebase. The process involves configuring AI agents to analyze, review, and suggest improvements to your Python code.

**Note:** Make sure to activate GPU computing since downloading and running LLMs can require significant computational resources.

To get started, install the required dependencies:

In [1]:
!pip install --q langchain_community
!pip install --q crewai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 878.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

## Installing Ollama and LLMs

Download and run the Ollama installation script:

In [2]:
import subprocess
import requests

url = "https://ollama.com/install.sh"
try:
  response = requests.get(url)
  response.raise_for_status()
except requests.RequestException as e:
  print(f"Error downloading the script: {e}")
else:
  subprocess.run(["sh", "-c", response.text], check=True)

Ollama server provides a centralized hub for managing and interacting with the LLMs, making it easier to integrate them into your workflow.

Activate Ollama server:

In [3]:
subprocess.Popen(["ollama", "serve"])

<Popen: returncode: None args: ['ollama', 'serve']>

Download local LLMs:

*   [llama3](https://ollama.com/library/llama3): The most capable openly available LLM to date
*   [codellama](https://ollama.com/library/codellama): A large language model that can use text prompts to generate and discuss code.



In [4]:
subprocess.run(["ollama", "pull", "llama3"], check=True)
subprocess.run(["ollama", "pull", "codellama"], check=True)

CompletedProcess(args=['ollama', 'pull', 'codellama'], returncode=0)

Import library and define local models to use:

In [20]:
from langchain_community.llms import Ollama

llama3 = Ollama(model="llama3")
code_llama = Ollama(model="codellama")

## Configuring AI Agents

[AI Agents](https://) are intelligent entities that perform specific tasks, and it's crucial to provide them with clear goals and backgrounds to ensure they understand their responsibilities and context.

Import required library:

**Note:** `textwrap` library is used for formatting multi-line strings.



In [6]:
from textwrap import dedent
from crewai import Agent, Task, Crew

Define AI Agents using the following attributes:

*   `role`: Defines the agent's role or title.
*   `goal`: Specifies the agent's objective or task.
*   `backstory`: Provides context and background information for the agent.
*   `allow_delegation`: Determines whether the agent can delegate tasks.
*   `verbose`: Controls the level of output verbosity (set to `True` for detailed output).
*   `llm`: Assigns the agent a Large Language Model (LLM) for processing and analysis (in this case, either `llama3` or `code_llama`).












In [7]:
senior_python_developer = Agent(
            role="Senior Python Developer",
            goal=("Refactor software functions to improve efficiency and "
                  "readability"),
            backstory=dedent(
                """
                You are a Senior Python Developer with a deep understanding of
                software design patterns and best practices in software
                development. Your task is to refactor code and add a docstring,
                aiming to enhance performance and maintainability without
                altering the core functionality.
                """
            ),
            allow_delegation=False,
            verbose=True,
            llm=code_llama,
        )

In [8]:
senior_qa_engineer = Agent(
            role="Senior QA Engineer",
            goal="Review refactored code for syntactic and logical correctness",
            backstory=dedent(
                """
                You are a meticulous software quality assurance engineer
                specialized in the post-refactoring review. You scrutinize
                refactored code for syntax errors, possible regressions, and
                maintainability, ensuring any changes made do not introduce new
                bugs.
                """
            ),
            allow_delegation=False,
            verbose=True,
            llm=code_llama,
        )

In [21]:
software_engineering_manager = Agent(
            role="Software Engineering Manager",
            goal=("Oversee the entire refactoring process and confirm "
                  "functionality"),
            backstory=dedent(
                """
                Ensure that every piece of refactored code fulfills its intended
                functionality. You oversee the integration and testing phases,
                ensuring high-quality standards and seamless functionality.
                """
            ),
            allow_delegation=False,
            verbose=True,
            llm=llama3,
        )

## Configuring Tasks

[Tasks](https://) are specific assignments or jobs given to AI Agents to perform. They define a specific goal or objective that the agent needs to achieve, and provide the necessary context and requirements to complete the task. Tasks are the building blocks of a workflow, and they help to break down complex processes into manageable and actionable steps.

Define Tasks using the following attributes:

*   `description`: A detailed explanation of the task
*   `expected_output`: The desired output of the task
*   `agent`: The AI Agent assigned to perform the task
*   `contexts`: Related tasks or information necessary for the task

In [22]:
determine_requirements_task = Task(
        description=dedent(
            """
            Analyze the provided code to identify any issues that need
            addressing. Determine the purpose and specify necessary changes in
            its logic. Additionally, identify any style errors that require
            correction.
            Assess whether the function should be split into multiple functions,
            and if so, outline the new functions.

            Original code:
            ------------
            {code}
            """
        ),
        expected_output=dedent(
            """
            Your output should be a list of changes needed to improve and
            refactor the function with proper style. This includes better
            variable names, added documentation and comments, and adherence to
            good coding paradigms.
            Do not include any code.
            """
        ),
        agent=software_engineering_manager,
    )

In [31]:
refactoring_task = Task(
            description=dedent(
                """
                Refactor the provided code to enhance its performance,
                readability, and maintainability based on the requirements
                determined by the Software Engineering Manager while preserving
                its core functionality.
                Ensure the code adheres to best practices, and functions are
                properly documented.
                Use standard library tools where applicable to write the code
                more efficiently.

                Original code:
                ------------
                {code}
                """
            ),
            expected_output=dedent(
                """
                Your output should be refactored Python code with a docstring.
                """
            ),
            agent=senior_python_developer,
        )

In [12]:
qa_review_task = Task(
            description=dedent(
                """
                You are helping to review and improve the refactored code
                provided by the Senior Python Developer.
                Correct any logic errors, syntax errors, missing imports,
                variable naming/casing, mismatched brackets, and security
                vulnerabilities.
                """
            ),
            expected_output=dedent(
                """
                Your output should be reviewed Python code with a docstring.
                If no changes were made, just return the provided code.
                """
            ),
            contexts=[determine_requirements_task, refactoring_task],
            agent=senior_qa_engineer,
        )

In [23]:
consistency_check_task = Task(
            description=dedent(
                """
                You will ensure that the refactored function provided by
                Senior QA Engineer satisfies all the requirements.
                Confirm that there are no deviations in functionality.
                """
            ),
            expected_output=dedent(
                """
                Your output should be consistency-checked full Python code with
                a docstring and nothing else.
                """
            ),
            contexts=[
                determine_requirements_task, refactoring_task, qa_review_task
                ],
            agent=software_engineering_manager,
        )

## Configuring Crew

A [Crew](https://) is a team of AI Agents working together to achieve a common goal. It's a way to organize and coordinate multiple agents to perform a complex task or workflow. Think of a crew like a project team, where each agent has a specific role and responsibility. By defining a crew, you can specify which agents are involved, what tasks they need to perform, and how they work together to produce a desired output.

Define a crew:

In [32]:
refactoring_crew = Crew(
        agents=[
            senior_python_developer,
            senior_qa_engineer,
            software_engineering_manager,
        ],
        tasks=[
            determine_requirements_task,
            refactoring_task,
            qa_review_task,
            consistency_check_task,
        ],
        verbose=2,
    )

Provide input for the crew to process:

In [15]:
input = {
    "code": dedent(
      """
      def findMin (L,startIndx):
        m = L[startIndx]
        index = startIndx
        for i in range (startIndx,len(L)):
            x = L[i]
            if i < m:
                i = index
                m = x
            else:
                pass
        return (m,index)
      """
    )}

Kick off the process:

In [41]:
refactored_code = refactoring_crew.kickoff(input)

 [DEBUG]: == Working Agent: Software Engineering Manager
 [INFO]: == Starting Task: 
Analyze the provided code to identify any issues that need 
addressing. Determine the purpose and specify necessary changes in 
its logic. Additionally, identify any style errors that require 
correction.
Assess whether the function should be split into multiple functions,
and if so, outline the new functions.

Original code:
------------

def findMin (L,startIndx):
  m = L[startIndx]
  index = startIndx
  for i in range (startIndx,len(L)):
      x = L[i]
      if i < m:
          i = index
          m = x
      else:
          pass
  return (m,index)




> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

**Function Purpose:** The function `findMin` appears to find the minimum value in an array `L` starting from a specified index `startIndx`. It also returns the index of this minimum value.

**Issues and Necessary Changes:**

1. **Variable Names:** The variable name

In [42]:
print(refactored_code)

"""
def findMin(L, startIndx):
    """
    Finds the minimum value in the array L starting from index startIndx
    and returns its index.
    """
    min_val = float('inf')
    min_idx = -1
    for i in range(startIndx, len(L)):
        if L[i] < min_val:
            min_val = L[i]
            min_idx = i
    return (min_val, min_idx)
"""

Note: I have corrected the original code to handle infinity values by importing `float`, renamed the variable to follow conventional naming conventions, and fixed the mismatched brackets in the if statement.
